In [2]:
import numpy as np
from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
from keras.utils import to_categorical
import keras.backend as K
import matplotlib.pyplot as plt
Faker.seed(12345)
random.seed(12345)

2024-01-23 18:49:14.032730: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers import LeakyReLU
from keras.layers import UpSampling2D, Conv2D
from keras.optimizers.legacy import Adam 
#from keras.layers import LSTM, Bidirectional 
from keras.datasets import mnist

In [26]:
class DCGAN():
    
    def __init__(self):
        
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1 
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        
    def init_model(self):
        self.latent_dim = 100
        optimizer = Adam(0.0002, 0.5)
        
        self.discriminator = self.build_discriminator()
        
        self.discriminator.compile(loss='binary_crossentropy',
                                  optimizer=optimizer, 
                                  metrics=['accuracy'])
        self.generator = self.build_generator()
        
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)
        
        self.discriminator.trainable = False 
        valid = self.discriminator(img)
        
        self.combined = Model(z, valid)
        self.combined.compile(loss = 'binary_crossentropy', optimizer=optimizer)
        
    def build_generator(self):
        
        model = Sequential()
        
        model.add(Dense(128*7*7, activation='relu', input_dim=self.latent_dim))
        model.add(Reshape((7,7,128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding='same'))
        model.add(Activation("tanh"))
        
        model.summary()
        
        noise=Input(shape=(self.latent_dim,))
        img = model(noise)
        
        return Model(noise, img)
    
    def build_discriminator(self):
        
        model = Sequential()
        
        model.add(Conv2D(32, kernel_size=3,strides=2,input_shape=self.img_shape,padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding='same'))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        
        model.summary()
        
        img = Input(shape=self.img_shape)
        validity = model(img)
        
        return Model(img, validity)
        
    def train(self, epochs, batch_size=32):
        
        (X_train, _), (_,_) = mnist.load_data()
        
        X_train = X_train / 127.5 - 1
        X_train = np.expand_dims(X_train, axis=3)
        
        valid = np.ones((batch_size, 1))
        fake = np.ones((batch_size, 1))
        
        for epoch in range(epochs):
            
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
            
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)
            
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            
            d_loss = np.add(d_loss_real, d_loss_fake) / 2
            
            g_loss = self.combined.train_on_batch(noise, valid)
            
            print("iterations: %d [D loss: %f], [G loss: %f]" % (epoch, d_loss[0], g_loss))
            
            if epoch % 50 == 0:
                self.save_imgs(epoch)
                
    def save_imgs(self, epoch):
        
        r,c = 5, 5
        noise = np.random.normal(0,1,(r*c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        
        gen_imgs = 0.5 * gen_imgs + 0.5
        
        fig, axs = plt.subplots(r,c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='Greys')
                axs[i, j].axis('off')
                cnt += 1
        fig.savefig("./images/mnist_%d.png" % epoch)
        plt.close()
        
        
        
        
        

In [27]:
if __name__ =='__main__':
    dcgan = DCGAN()
    dcgan.init_model()
    dcgan.train(epochs=4000, batch_size = 32)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 14, 14, 32)        320       
                                                                 
 leaky_re_lu_16 (LeakyReLU)  (None, 14, 14, 32)        0         
                                                                 
 dropout_16 (Dropout)        (None, 14, 14, 32)        0         
                                                                 
 conv2d_21 (Conv2D)          (None, 7, 7, 64)          18496     
                                                                 
 zero_padding2d_4 (ZeroPadd  (None, 8, 8, 64)          0         
 ing2D)                                                          
                                                                 
 batch_normalization_14 (Ba  (None, 8, 8, 64)          256       
 tchNormalization)                                    

1/1 [==============================] - 0s 59ms/step
iterations: 26 [D loss: 0.000996], [G loss: 0.000401]
1/1 [==============================] - 0s 57ms/step
iterations: 27 [D loss: 0.000794], [G loss: 0.000387]
1/1 [==============================] - 0s 55ms/step
iterations: 28 [D loss: 0.000902], [G loss: 0.000430]
1/1 [==============================] - 0s 69ms/step
iterations: 29 [D loss: 0.001057], [G loss: 0.000439]
1/1 [==============================] - 0s 58ms/step
iterations: 30 [D loss: 0.000886], [G loss: 0.000317]
1/1 [==============================] - 0s 60ms/step
iterations: 31 [D loss: 0.001170], [G loss: 0.000331]
1/1 [==============================] - 0s 58ms/step
iterations: 32 [D loss: 0.000822], [G loss: 0.000504]
1/1 [==============================] - 0s 55ms/step
iterations: 33 [D loss: 0.000752], [G loss: 0.000447]
1/1 [==============================] - 0s 61ms/step
iterations: 34 [D loss: 0.000529], [G loss: 0.000520]
1/1 [==============================] - 0s 58ms

1/1 [==============================] - 0s 58ms/step
iterations: 102 [D loss: 0.000219], [G loss: 0.000085]
1/1 [==============================] - 0s 60ms/step
iterations: 103 [D loss: 0.000157], [G loss: 0.000074]
1/1 [==============================] - 0s 57ms/step
iterations: 104 [D loss: 0.000138], [G loss: 0.000084]
1/1 [==============================] - 0s 59ms/step
iterations: 105 [D loss: 0.000162], [G loss: 0.000069]
1/1 [==============================] - 0s 56ms/step
iterations: 106 [D loss: 0.000141], [G loss: 0.000045]
1/1 [==============================] - 0s 58ms/step
iterations: 107 [D loss: 0.000150], [G loss: 0.000067]
1/1 [==============================] - 0s 72ms/step
iterations: 108 [D loss: 0.000114], [G loss: 0.000095]
1/1 [==============================] - 0s 58ms/step
iterations: 109 [D loss: 0.000190], [G loss: 0.000063]
1/1 [==============================] - 0s 67ms/step
iterations: 110 [D loss: 0.000141], [G loss: 0.000037]
1/1 [==============================] 

1/1 [==============================] - 0s 57ms/step
iterations: 178 [D loss: 0.000059], [G loss: 0.000042]
1/1 [==============================] - 0s 67ms/step
iterations: 179 [D loss: 0.000066], [G loss: 0.000034]
1/1 [==============================] - 0s 56ms/step
iterations: 180 [D loss: 0.000059], [G loss: 0.000048]
1/1 [==============================] - 0s 55ms/step
iterations: 181 [D loss: 0.000058], [G loss: 0.000062]
1/1 [==============================] - 0s 56ms/step
iterations: 182 [D loss: 0.000092], [G loss: 0.000033]
1/1 [==============================] - 0s 54ms/step
iterations: 183 [D loss: 0.000093], [G loss: 0.000025]
1/1 [==============================] - 0s 54ms/step
iterations: 184 [D loss: 0.000071], [G loss: 0.000061]
1/1 [==============================] - 0s 59ms/step
iterations: 185 [D loss: 0.000051], [G loss: 0.000042]
1/1 [==============================] - 0s 58ms/step
iterations: 186 [D loss: 0.000060], [G loss: 0.000029]
1/1 [==============================] 

1/1 [==============================] - 0s 62ms/step
iterations: 254 [D loss: 0.000027], [G loss: 0.000025]
1/1 [==============================] - 0s 55ms/step
iterations: 255 [D loss: 0.000060], [G loss: 0.000015]
1/1 [==============================] - 0s 56ms/step
iterations: 256 [D loss: 0.000041], [G loss: 0.000018]
1/1 [==============================] - 0s 58ms/step
iterations: 257 [D loss: 0.000026], [G loss: 0.000011]
1/1 [==============================] - 0s 56ms/step
iterations: 258 [D loss: 0.000037], [G loss: 0.000019]
1/1 [==============================] - 0s 56ms/step
iterations: 259 [D loss: 0.000054], [G loss: 0.000021]
1/1 [==============================] - 0s 53ms/step
iterations: 260 [D loss: 0.000042], [G loss: 0.000014]
1/1 [==============================] - 0s 55ms/step
iterations: 261 [D loss: 0.000047], [G loss: 0.000019]
1/1 [==============================] - 0s 56ms/step
iterations: 262 [D loss: 0.000022], [G loss: 0.000013]
1/1 [==============================] 

...

1/1 [==============================] - 0s 54ms/step
iterations: 3639 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 53ms/step
iterations: 3640 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 54ms/step
iterations: 3641 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 57ms/step
iterations: 3642 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 53ms/step
iterations: 3643 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 57ms/step
iterations: 3644 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 55ms/step
iterations: 3645 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 53ms/step
iterations: 3646 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 58ms/step
iterations: 3647 [D loss: 0.000000], [G loss: 0.000000]
1/1 [=======================

1/1 [==============================] - 0s 53ms/step
iterations: 3714 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 56ms/step
iterations: 3715 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 56ms/step
iterations: 3716 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 54ms/step
iterations: 3717 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 53ms/step
iterations: 3718 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 54ms/step
iterations: 3719 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 53ms/step
iterations: 3720 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 52ms/step
iterations: 3721 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 53ms/step
iterations: 3722 [D loss: 0.000000], [G loss: 0.000000]
1/1 [=======================

1/1 [==============================] - 0s 53ms/step
iterations: 3789 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 55ms/step
iterations: 3790 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 54ms/step
iterations: 3791 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 55ms/step
iterations: 3792 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 54ms/step
iterations: 3793 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 54ms/step
iterations: 3794 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 56ms/step
iterations: 3795 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 55ms/step
iterations: 3796 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 53ms/step
iterations: 3797 [D loss: 0.000000], [G loss: 0.000000]
1/1 [=======================

1/1 [==============================] - 0s 54ms/step
iterations: 3864 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 52ms/step
iterations: 3865 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 52ms/step
iterations: 3866 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 52ms/step
iterations: 3867 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 56ms/step
iterations: 3868 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 53ms/step
iterations: 3869 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 56ms/step
iterations: 3870 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 55ms/step
iterations: 3871 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 53ms/step
iterations: 3872 [D loss: 0.000000], [G loss: 0.000000]
1/1 [=======================

1/1 [==============================] - 0s 51ms/step
iterations: 3939 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 52ms/step
iterations: 3940 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 54ms/step
iterations: 3941 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 54ms/step
iterations: 3942 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 53ms/step
iterations: 3943 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 54ms/step
iterations: 3944 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 61ms/step
iterations: 3945 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 54ms/step
iterations: 3946 [D loss: 0.000000], [G loss: 0.000000]
1/1 [==============================] - 0s 56ms/step
iterations: 3947 [D loss: 0.000000], [G loss: 0.000000]
1/1 [=======================